In [2]:
import pandas as pd
import optuna
import matplotlib.pyplot as plt

from calibrate import read_data, calc_multi_error
from plot import to_df, get_pareto_front, get_contour_plot, get_parallel_coordinate_plot

%matplotlib inline

In [8]:
#storage = "sqlite:///calibration-2020-06-19.db"
storage = "sqlite:///calibration.db"
print("Studies:")
for s in optuna.study.get_all_study_summaries(storage):
    print(s.study_name, s.datetime_start, s.n_trials, "trials")

Studies:
multi 2020-06-19 11:13:00.526616 371 trials


In [9]:
study = optuna.multi_objective.load_study("multi", storage)

C:\tools\Anaconda3\lib\site-packages\optuna\_experimental.py:90: ExperimentalWarning:

load_study is experimental (supported from v1.4.0). The interface can change in the future.

C:\tools\Anaconda3\lib\site-packages\optuna\_experimental.py:111: ExperimentalWarning:

RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.

C:\tools\Anaconda3\lib\site-packages\optuna\_experimental.py:111: ExperimentalWarning:

MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.



In [ ]:
unc = optuna.load_study("unconstrained", storage)
uf = to_df(unc)
print(uf.loc[uf.value.idxmin()])
optuna.visualization.plot_parallel_coordinate(unc)

In [12]:
tf, fig = get_pareto_front(study, hover=["ciCorrection", "offset", "hospital"], color="dz")
fig.show()

Best avg.
number                            290
dz                            2.71221
error_cases                 0.0490776
error_critical               0.646242
error_sick                   0.522366
peak              2020-03-25 00:00:00
ciCorrection                 0.339954
hospital                       1.0544
offset                             -1
error_hospital                1.16861
error_total                   1.26676
Name: 289, dtype: object


In [ ]:
number = 221

start="2020-03-01"
end="2020-06-01"
f = "output-calibration/%d/infections.txt" % number
cmp = "output-compare/infections.txt"

df, hospital, rki = read_data(f, "Berlin", "berlin-hospital.csv", "berlin-cases.csv")

cmp_res = calc_multi_error(cmp, "Berlin", start, end)
cmp, _, _ = read_data(cmp, "Berlin", "berlin-hospital.csv", "berlin-cases.csv")

df = df[(df.date >= start) & (df.date <= end)]
hospital = hospital[(hospital.Datum >= start) & (hospital.Datum <= end)]
rki = rki[(rki.date >= start) & (rki.date <= end)]

print(cmp_res)
tf[tf.number == number]

In [ ]:
ax = df.plot.scatter(x="date", y=["cases"], label=["Auto Calibrated"], logy=True, figsize=(16, 8))
cmp.plot.scatter(x="date", y=["cases"], color="green", label=["Manual"], ax=ax)
rki.plot.scatter(x="date", y=["cases"], color="orange", label=["RKI Cases"], ax=ax)
plt.title("Cases")

In [ ]:
ax = df.plot(x="date", y=["casesSmoothed"], label=["Auto calibrated"], logy=True, figsize=(16, 8))
rki.plot(x="date", y=["casesSmoothed"], label=["RKI cases"], ax=ax)
cmp.plot(x="date", y=["casesSmoothed"], label=["Manual"], ax=ax)
plt.title("Smoothed cases")

In [ ]:
ax = df.plot(x="date", y=["nSeriouslySick", "nCritical"], logy=True, figsize=(16, 8))
hospital.plot(x="Datum", y=["Stationäre Behandlung", "Intensivmedizin"], logy=True, ax=ax)

plt.title("Hospitalization")

In [13]:
get_parallel_coordinate_plot(study)

In [6]:
get_contour_plot(study, "alpha", "ciCorrection")

C:\tools\Anaconda3\lib\site-packages\optuna\_experimental.py:111: ExperimentalWarning:

FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.

